In [1]:
1+1

2

In [2]:
from pathlib import Path
import os
import time
import numpy as np
import torch

import warnings
warnings.filterwarnings('ignore')

from data_loader import *
from model import *
from test import *


# pre-process
def pre_proc(tau, block):
    '''log(tau), 95%'''
    tau_sum = tau.sum(axis=(-1,-2,-3))
    limit = np.percentile(tau_sum, 95)
    bln = np.ones(len(tau), 'bool')#tau_sum <= limit
    tau = np.log(tau)
    return (tau[bln],  block[bln])

def toF_proc(tau):
    '''transfer data derived from pre_proc to F=exp(-tau)'''
    tau = np.exp(-np.exp(tau))
    return tau

def load_FGPA(FGPA, train_ousize, DM_param):
    nx, ny, nz = (DM_param.pix / train_ousize).astype('int')
    FGPA = FGPA.reshape(nx, train_ousize[0], ny, train_ousize[1], DM_param.pix)\
        .transpose(0, 2, 1, 3, 4).reshape(-1, train_ousize[0], train_ousize[1], DM_param.pix)

    x = (np.arange(nx)*(train_ousize[0]) + (train_ousize[0]-1)/2)
    y = (np.arange(ny)*(train_ousize[1]) + (train_ousize[1]-1)/2)
    z = (np.arange(nz)*(train_ousize[2]) + (train_ousize[2]-1)/2)
    
    cx = x.repeat(ny*nz).reshape(nx,ny,nz).transpose(0,1,2).flatten()
    cy = y.repeat(nz*nx).reshape(ny,nz,nx).transpose(2,0,1).flatten()
    cz = z.repeat(nx*ny).reshape(nz,nx,ny).transpose(1,2,0).flatten()
    block = np.array([cx, cy, cz]).T.reshape(-1, nz, 3)

    return FGPA




# Path and data file name
folder  = Path.cwd().parent / 'Illustris1'
DM_name = ['deltaDM_Illustris1_L75_N600_v2.fits', 
            'vx_DM_Illustris1_L75_N600.fits',
            'vy_DM_Illustris1_L75_N600.fits',
            'vz_DM_Illustris1_L75_N600.fits']
ske_name = 'spectra_Illustris1_N600_xaxis.npy'
FGPA_name = 'tau_normed_FGPA_smooth250kpc_therbr_RSD_Illustris-1_snap68_L75_N600_cic_xaxis.fits'



# hyper parameters
train_insize = np.array([11, 11, 141]) # x, y, z respctively
train_ousize = np.array([1, 1, 75]) # x, y, z respctively
test_batch = 50

In [3]:
localtime_n = ['2020-12-24 23:42:28']
localtime_i = localtime_n[0]
localtime = time.strptime(localtime_i, '%Y-%m-%d %H:%M:%S')


# device used to train the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using device:', torch.cuda.get_device_name(device=device.index))


# load dark matter data
print('Loading dark matter...')
DM_general = load_DM(folder, DM_name)
DM_general = DM_general.transpose(0,3,2,1)
DM_general = DM_general[[0,3,2,1]]
# basic paramters
DM_param.pix  = len(DM_general[0])
DM_param.len  = 75 # in Mpc/h
DM_param.reso = DM_param.len / DM_param.pix # in Mpc/h
# test
if DM_general.shape[1]<train_insize.min():
    raise ValueError('DarkMatter cube size',
        DM_general.shape, 'is too small for train size', train_insize, '.')
DM_general = torch.tensor(DM_general).float()


# load skewers
print('Loading skewers...')
ske, block = load_skewers(folder, ske_name, train_ousize, DM_param)
#ske_FGPA, block = load_skewers(folder, FGPA_name, train_ousize, DM_param)
skeFGPA_file = fits.open(folder/FGPA_name)
ske_FGPA = skeFGPA_file[0].data
skeFGPA_file.close()
ske_FGPA = ske_FGPA.transpose(2,1,0)
ske_FGPA = load_FGPA(ske_FGPA, train_ousize, DM_param)
# basic parameters
ske_len = int(ske.shape[-1])


# divide the sample to training, validation set, and test set.
print('Setting test set...')
'''
with open("id_seperate/id_seperate_%s.txt"\
          %time.strftime("%Y-%m-%d_%H:%M:%S", localtime), "r") as f:
    aa = f.readlines()
    id_seperate = np.array(list(aa[0][::3])).astype('int')
    del aa
f.close()
'''
id_seperate = (np.concatenate((np.ones(36000), np.zeros(360000-36000)))*3).astype(int)
np.random.shuffle(id_seperate)

test_ske, test_block = load_test(ske, block, id_seperate,
                                 train_ousize, test_batch, pre_proc)
test_FGPA, test_block = load_test(ske_FGPA, block, id_seperate,
                                 train_ousize, test_batch, pre_proc)
del id_seperate

Using device: Quadro P6000
Loading dark matter...
Loading skewers...
Setting test set...


In [4]:
# this parameter deviates the PDF of ML F and hydro
# why so large difference? maybe train on normalized skewers
normalization_outp, normalization_test

NameError: name 'normalization_outp' is not defined

In [4]:
from scipy import optimize
def norm_specs(skewers,redshift):
    tauevo = 0.001845*(1+redshift)**3.924 #optical depth corresponding to the mean-flux of Faucher-Giguere et al. 2008 
    fobs = np.exp(-tauevo)
    def fun(a): return np.mean(np.exp(-a*skewers)-fobs)
    a = optimize.newton(fun, 1)    
    return a
rs = 2.0020281392528516 #redshift of the snapshot


# load model
print('Loading model...')
model = get_residual_network().float().to(device)
model.load_state_dict(torch.load('params/params_%s.pkl'\
        %time.strftime("%Y-%m-%d_%H:%M:%S", localtime)))
'''
model.load_state_dict(torch.load('params/HyPhy_%s'\
        %time.strftime("%Y-%m-%d_%H:%M:%S", localtime)))
'''


# loss
criterion = nn.L1Loss()


# record starr time
start_time = time.time()


# start test
print('Begin testing...')
test_outp, test_losses = test(test_ske, test_block, DM_general, DM_param,
                        test_batch, train_insize, model, criterion, device, start_time)

print("Test Summary: ")
print("\tTest loss: {}".format(test_losses))

# restore test skewers
print('Restoring test skewers...')
nz = (ske_len/train_ousize[2]).astype('int')
test_outp = test_outp.reshape(-1, nz, train_ousize[0],
                            train_ousize[1], train_ousize[2])\
                            .transpose(0, 2, 3, 1, 4).reshape(-1, ske_len)
test_ske = test_ske.numpy().reshape(-1, nz, train_ousize[0],
                            train_ousize[1], train_ousize[2])\
                            .transpose(0, 2, 3, 1, 4).reshape(-1, ske_len)
test_FGPA = test_FGPA.numpy().reshape(-1, nz, train_ousize[0],
                            train_ousize[1], train_ousize[2])\
                            .transpose(0, 2, 3, 1, 4).reshape(-1, ske_len)

tau = np.exp(test_ske)
F_hydro = np.exp(-tau)
F_sum = F_hydro.sum(axis=1)
limit = np.percentile(F_sum, 10)
bln = F_sum >= limit
tau = tau[bln]
normalization = norm_specs(tau,rs) #determine the normalization constant on the whole tau box
tau_normed = normalization*tau #normalized optical depth
test_ske = np.exp(-normalization*tau) #normalized flux

tau = np.exp(test_outp[bln])
normalization = norm_specs(tau,rs) #determine the normalization constant on the whole tau box
tau_normed = normalization*tau #normalized optical depth
test_outp = np.exp(-normalization*tau) #normalized flux

tau = np.exp(test_FGPA[bln])
normalization = norm_specs(tau,rs) #determine the normalization constant on the whole tau box
tau_normed = normalization*tau #normalized optical depth
test_FGPA = np.exp(-normalization*tau) #normalized flux

Loading model...
Begin testing...
Step [ 100/5760] Loss: 0.5311, Time: 14.5992
Step [ 200/5760] Loss: 0.4116, Time: 29.7110
Step [ 300/5760] Loss: 0.5083, Time: 44.9756
Step [ 400/5760] Loss: 0.4794, Time: 59.6375
Step [ 500/5760] Loss: 0.4579, Time: 74.2661
Step [ 600/5760] Loss: 0.4573, Time: 89.4825
Step [ 700/5760] Loss: 0.4263, Time: 104.1855
Step [ 800/5760] Loss: 0.8187, Time: 119.0696
Step [ 900/5760] Loss: 0.3824, Time: 133.8021
Step [1000/5760] Loss: 0.4776, Time: 148.5945
Step [1100/5760] Loss: 0.4278, Time: 163.8051
Step [1200/5760] Loss: 0.3739, Time: 178.8925
Step [1300/5760] Loss: 0.4636, Time: 194.0164
Step [1400/5760] Loss: 0.3488, Time: 209.5528
Step [1500/5760] Loss: 0.5367, Time: 224.3768
Step [1600/5760] Loss: 0.4038, Time: 239.1475
Step [1700/5760] Loss: 0.7081, Time: 253.9317
Step [1800/5760] Loss: 0.7611, Time: 268.7659
Step [1900/5760] Loss: 0.5453, Time: 283.6090
Step [2000/5760] Loss: 0.4230, Time: 298.5796
Step [2100/5760] Loss: 0.6926, Time: 313.1705
Step [

In [5]:
'''
test_outp = toF_proc(test_outp)
test_ske  = toF_proc(test_ske)
test_FGPA = toF_proc(test_FGPA)
'''
test_coord = test_block.reshape(-1, nz, 3)[:, 0, 0:2].T.reshape(2, 1, 1, -1)
xcoor = np.arange(train_ousize[0]) - np.arange(train_ousize[0]).mean()
ycoor = np.arange(train_ousize[1]) - np.arange(train_ousize[1]).mean()
mesh  = np.expand_dims(np.array(np.meshgrid(xcoor, ycoor)), -1)
test_block = (test_coord + mesh).transpose(3,2,1,0).reshape(-1, 2).astype(int)
del test_coord, xcoor, ycoor, mesh


print('Plotting example skewers...')
# generate comparison images
folder_outp = Path.cwd()/'test_figs'/('%s_x'\
        %time.strftime("%Y-%m-%d_%H:%M:%S_FGPA", localtime))
if not os.path.exists(folder_outp):
    os.makedirs(folder_outp)


from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
z = 2.0
h= 0.704
cosmo = FlatLambdaCDM(H0=100.0*h, Om0=0.2726, Ob0=0.0456)
Hz = cosmo.H(z)
a = 1.0 / (1.0 + z)
v_end  = (a * Hz * 75/h*u.Mpc).to(u.km / u.s).value
F_mean = np.array([test_ske.mean(), test_outp.mean(), test_FGPA.mean()])
print(F_mean)

nrange = min(len(test_ske), 200)
test_sp = np.arange(len(test_ske))
np.random.seed(99)
np.random.shuffle(test_sp)
test_sp1 = test_sp[:int(nrange)].astype('int')
test_sp2 = test_sp[int(nrange):].astype('int')

bins = int(15)
accuracy = AverageMeter()
rela_err = AverageMeter()
accu_arr = np.zeros(shape=(len(test_ske), 2))
erro_arr = np.zeros(shape=(len(test_ske), 2))
oneDPS   = np.zeros(shape=(4, len(test_ske), bins))


# loop
import datetime
from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('Example_Skewers.pdf') as pdf:
    for i, ii in enumerate(test_sp1):
        print('Plotting {:{}d}/{}, y{:03d}z{:03d}.png...'\
                .format((i+1), int(np.log10(nrange)+1), nrange,
                        test_block[ii,0], test_block[ii,1]))

        test_block_i = test_block[ii]
        test_outp_i = test_outp[ii]
        test_ske_i = test_ske[ii]
        test_FGPA_i = test_FGPA[ii]
        test_DM_i = DM_general[0, test_block_i[0], test_block_i[1], :].numpy()

        stat_i, fig = test_accuracy(test_block_i, test_outp_i, test_ske_i, test_FGPA_i,
                          test_DM_i, F_mean, v_end, folder_outp, bins, plot=True)
        accuracy_i, rela_err_i = stat_i[[4,5]]
        accuracy.update(accuracy_i, 1)
        rela_err.update(rela_err_i, 1)
        accu_arr[ii] = accuracy_i
        erro_arr[ii] = rela_err_i
        oneDPS[:,ii] = stat_i[0], stat_i[1], stat_i[2], stat_i[3]
    
        pdf.savefig(fig)
        
    d = pdf.infodict()
    d['Title'] = 'Example Skewers'
    d['Author'] = 'Yi Kang'
    #d['Subject'] = 'How to create a multipage pdf file and set its metadata'
    #d['Keywords'] = 'PdfPages multipage keywords author title subject'
    d['CreationDate'] = datetime.datetime.today()
    d['ModDate'] = datetime.datetime.today()
    
print('Measuring accuracy of the rest skewers...')
for i, ii in enumerate(test_sp2):

    test_block_i = test_block[ii]
    test_outp_i = test_outp[ii]
    test_ske_i = test_ske[ii]
    test_FGPA_i = test_FGPA[ii]
    test_DM_i = DM_general[0, test_block_i[0], test_block_i[1], :].numpy()

    stat_i = test_accuracy(test_block_i, test_outp_i, test_ske_i, test_FGPA_i,
                      test_DM_i, F_mean, v_end, folder_outp, bins, plot=False)
    accuracy_i, rela_err_i = stat_i[[4,5]]
    accuracy.update(accuracy_i, 1)
    rela_err.update(rela_err_i, 1)
    accu_arr[ii] = accuracy_i
    erro_arr[ii] = rela_err_i
    oneDPS[:,ii] = stat_i[0], stat_i[1], stat_i[2], stat_i[3]


print('Plotting average 1DPS and PDF...')
oneDPS_backup = oneDPS
oneDPS = oneDPS.mean(axis=1)
oneDPS = oneDPS[~np.isnan(oneDPS)].reshape(4,-1)
bln = oneDPS[0]<0.1
accuracy_gen = np.abs((oneDPS[1]-oneDPS[2])/oneDPS[2])[bln].mean()
rela_err_gen = np.abs((oneDPS[1]-oneDPS[2])/oneDPS[2])[bln].std()
accuracy_FGPA = np.abs((oneDPS[3]-oneDPS[2])/oneDPS[2])[bln].mean()
rela_err_FGPA = np.abs((oneDPS[3]-oneDPS[2])/oneDPS[2])[bln].std()

outp_hist, F_hist = np.histogram(test_outp, bins=np.arange(0,1.05,0.05))
test_hist, F_hist = np.histogram(test_ske,  bins=np.arange(0,1.05,0.05))
FGPA_hist, F_hist = np.histogram(test_FGPA, bins=np.arange(0,1.05,0.05))
outp_hist = np.append(outp_hist, outp_hist[-1]) / len(test_ske)
test_hist = np.append(test_hist, test_hist[-1]) / len(test_ske)
FGPA_hist = np.append(FGPA_hist, FGPA_hist[-1]) / len(test_ske)
bln = (F_hist>=0.1) & (F_hist<0.9)
accuracy_hist = np.abs((outp_hist-test_hist)/test_hist)[bln].mean()
rela_err_hist = np.abs((outp_hist-test_hist)/test_hist)[bln].std()
m_hist_FGPA = np.abs((FGPA_hist-test_hist)/test_hist)[bln].mean()
s_hist_FGPA = np.abs((FGPA_hist-test_hist)/test_hist)[bln].std()

Plotting example skewers...
[0.87123859 0.87123901 0.87123972]
Plotting   1/200, y117z387.png...
Plotting   2/200, y083z171.png...
Plotting   3/200, y249z282.png...
Plotting   4/200, y150z391.png...
Plotting   5/200, y166z362.png...
Plotting   6/200, y240z115.png...
Plotting   7/200, y152z362.png...
Plotting   8/200, y041z477.png...
Plotting   9/200, y056z543.png...
Plotting  10/200, y310z357.png...
Plotting  11/200, y390z428.png...
Plotting  12/200, y143z472.png...
Plotting  13/200, y229z531.png...
Plotting  14/200, y512z254.png...
Plotting  15/200, y000z235.png...
Plotting  16/200, y378z140.png...
Plotting  17/200, y045z232.png...
Plotting  18/200, y316z153.png...
Plotting  19/200, y538z324.png...
Plotting  20/200, y256z403.png...
Plotting  21/200, y137z404.png...
Plotting  22/200, y524z194.png...
Plotting  23/200, y170z193.png...
Plotting  24/200, y215z387.png...
Plotting  25/200, y084z503.png...
Plotting  26/200, y232z411.png...
Plotting  27/200, y179z113.png...
Plotting  28/200, y

UnboundLocalError: local variable 'fig' referenced before assignment

In [ ]:

fig, axes = plt.subplots(2,2,figsize=(15,11))

p0 = axes[0,0].hist(accu_arr[:,0], label='ML', bins=np.arange(0, 1.7, 0.1))
axes[0,0].set_xlim(-0.1, 1.8)
axes[0,0].set_ylim(axes[0,0].get_ylim())
p1 = axes[0,0].vlines(x=np.median(accu_arr[:,0]), ymin=0, ymax=9999, linestyle='--')
p9 = axes[0,0].hist(accu_arr[:,1], label='FGPA', color='tab:green', bins=np.arange(0, 1.7, 0.1), alpha=0.5)
axes[0,0].set_xlabel('accuracy $m$', fontsize=14)
axes[0,0].set_title('pdf of $m$', fontsize=14)
axes[0,0].tick_params(labelsize=12, direction='in')
customs = [p1, 
          Line2D([0], [0], marker='o', color='w',
                  markerfacecolor='k', markersize=5),
          p9]
axes[0,0].legend(customs, ['median $m=%.4f$'%np.median(accu_arr),
                        '$N=%d$'%len(accu_arr)], fontsize=12, loc=1)

axes[0,1].hist(erro_arr[0], color='grey', bins=np.arange(0, 1.7, 0.1))
axes[0,1].set_xlim(-0.1, 1.8)
axes[0,1].set_ylim(axes[0,1].get_ylim())
p2 = axes[0,1].vlines(x=erro_arr.mean(), ymin=0, ymax=9999, linestyle='--')
axes[0,1].set_xlabel('error $s$', fontsize=14)
axes[0,1].set_title('pdf of $s$', fontsize=14)
axes[0,1].tick_params(labelsize=12, direction='in')
customs = [p2, 
          Line2D([0], [0], marker='o', color='w',
                  markerfacecolor='k', markersize=5)]
axes[0,1].legend(customs, ['average $s=%.4f$'%erro_arr.mean(),
                        '$N=%d$'%len(erro_arr)], fontsize=12, loc=1)

p3, = axes[1,0].plot(oneDPS[0], oneDPS[1], label='ML')
p4, = axes[1,0].plot(oneDPS[0], oneDPS[2], label='Hydro', alpha=0.5)
p7, = axes[1,0].plot(oneDPS[0], oneDPS[3], label='FGPA', alpha=0.5)
axes[1,0].set_xlabel(r'$k\ (\mathrm{s/km})$', fontsize=14)
axes[1,0].set_ylabel(r'$kP_\mathrm{1D}/\pi$', fontsize=14)
axes[1,0].set_xscale('log')
axes[1,0].set_yscale('log')
axes[1,0].set_ylim(axes[1,0].get_ylim())
axes[1,0].vlines(x=0.1, ymin=1e-8, ymax=1e8)
axes[1,0].set_title('Average 1DPS', fontsize=14)
axes[1,0].tick_params(labelsize=12, direction='in', which='both')
customs = [p3,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p7,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p4]
axes[1,0].legend(customs, [p3.get_label(), '$m=%.3f$'%accuracy_gen, '$s=%.3f$'%rela_err_gen, 
                    p7.get_label(),'$m=%.3f$'%accuracy_FGPA, '$s=%.3f$'%rela_err_FGPA,
                    p4.get_label()], fontsize=12)

p5, = axes[1,1].step(F_hist, outp_hist, where='post', label='ML')
p6, = axes[1,1].step(F_hist, test_hist, where='post', label='Hydro', alpha=0.5)
p8, = axes[1,1].step(F_hist, FGPA_hist, where='post', label='FGPA', alpha=0.5)
axes[1,1].set_xlabel(r'$F$', fontsize=18)
axes[1,1].set_ylabel(r'Counts', fontsize=18)
axes[1,1].set_xlim([0, 1])
axes[1,1].set_ylim(bottom=0)
axes[1,1].vlines(x=0.1, ymin=0, ymax=6000)
axes[1,1].vlines(x=0.9, ymin=0, ymax=6000)
axes[1,1].set_title('Average PDF of $F$', fontsize=14)
axes[1,1].tick_params(labelsize=12, direction='in')
customs = [p5,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p8,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p6]
axes[1,1].legend(customs, [p5.get_label(), '$m=%.3f$'%accuracy_hist, '$s=%.3f$'%rela_err_hist,
                    p8.get_label(),'$m=%.3f$'%m_hist_FGPA, '$s=%.3f$'%s_hist_FGPA,
                    p6.get_label()], fontsize=12)

#plt.savefig(folder_outp / ('average.png'), dpi=300, bbox_inches='tight') 

In [ ]:
examples = 4
vaxis = np.arange(ske_len)/ske_len * v_end
order = np.random.randint(len(test_ske), size=examples)

fig, axes = plt.subplots(examples,1,figsize=(12,examples*2.4))

for i in range(examples):
    axes[i].plot(vaxis, test_outp[order[i]], label='ML',  alpha=0.7)
    axes[i].plot(vaxis, test_FGPA[order[i]], label='FGPA', color='tab:green', alpha=0.5)
    axes[i].plot(vaxis, test_ske[order[i]], label='Hydro', color='tab:orange', alpha=0.7)
    axes[i].set_xlim([0, max(vaxis)])
    axes[i].set_ylim([0,1])
    axes[i].set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8])
    axes[i].set_ylabel(r'$F$', fontsize=18, labelpad=0.5)
    if i != int(examples-1):
        axes[i].set_xticklabels([])
axes[0].set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
axes[-1].set_xlabel(r'$v$ (km/s)', fontsize=18, labelpad=0.5)
axes[-1].legend(fontsize=18, bbox_to_anchor=(1.25,2.75))
plt.subplots_adjust(hspace=0)
#plt.savefig(folder_outp / ('examples.png'), dpi=600, bbox_inches='tight') 

In [ ]:
final_deviation = (oneDPS[2] - oneDPS[1])[0]

In [ ]:
oneDPS_backup.shape

In [ ]:
oneDPS_deviation = (oneDPS_backup[2] - oneDPS_backup[1])

In [ ]:
bln = oneDPS_deviation[:,0] > final_deviation

In [ ]:
tau = np.exp(test_ske)
F_hydro = np.exp(-tau)
F_sum = F_hydro.sum(axis=1)
limit = np.percentile(F_sum, 20)
bln = F_sum >= limit
tau = tau[bln]
normalization = norm_specs(tau,rs) #determine the normalization constant on the whole tau box
tau_normed = normalization*tau #normalized optical depth
test_ske = np.exp(-normalization*tau) #normalized flux

tau = np.exp(test_outp[bln])
normalization = norm_specs(tau,rs) #determine the normalization constant on the whole tau box
tau_normed = normalization*tau #normalized optical depth
test_outp = np.exp(-normalization*tau) #normalized flux

tau = np.exp(test_FGPA[bln])
normalization = norm_specs(tau,rs) #determine the normalization constant on the whole tau box
tau_normed = normalization*tau #normalized optical depth
test_FGPA = np.exp(-normalization*tau) #normalized flux
'''
test_outp = toF_proc(test_outp)
test_ske  = toF_proc(test_ske)
test_FGPA = toF_proc(test_FGPA)
'''
test_coord = test_block.reshape(-1, nz, 3)[:, 0, 0:2].T.reshape(2, 1, 1, -1)
xcoor = np.arange(train_ousize[0]) - np.arange(train_ousize[0]).mean()
ycoor = np.arange(train_ousize[1]) - np.arange(train_ousize[1]).mean()
mesh  = np.expand_dims(np.array(np.meshgrid(xcoor, ycoor)), -1)
test_block = (test_coord + mesh).transpose(3,2,1,0).reshape(-1, 2).astype(int)
del test_coord, xcoor, ycoor, mesh


print('Plotting example skewers...')
# generate comparison images
folder_outp = Path.cwd()/'test_figs'/('%s_x'\
        %time.strftime("%Y-%m-%d_%H:%M:%S_FGPA", localtime))
if not os.path.exists(folder_outp):
    os.makedirs(folder_outp)


from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
z = 2.0
h= 0.704
cosmo = FlatLambdaCDM(H0=100.0*h, Om0=0.2726, Ob0=0.0456)
Hz = cosmo.H(z)
a = 1.0 / (1.0 + z)
v_end  = (a * Hz * 75/h*u.Mpc).to(u.km / u.s).value
F_mean = np.array([test_ske.mean(), test_outp.mean(), test_FGPA.mean()])
print(F_mean)

nrange = min(len(test_ske), 50)
test_sp = np.arange(len(test_ske))
np.random.seed(99)
np.random.shuffle(test_sp)
test_sp1 = test_sp[:int(nrange)].astype('int')
test_sp2 = test_sp[int(nrange):].astype('int')

bins = int(15)
accuracy = AverageMeter()
rela_err = AverageMeter()
accu_arr = np.zeros(shape=(len(test_ske), 2))
erro_arr = np.zeros(shape=(len(test_ske), 2))
oneDPS   = np.zeros(shape=(4, len(test_ske), bins))


# loop
for i, ii in enumerate(test_sp1):
    print('Plotting {:{}d}/{}, y{:03d}z{:03d}.png...'\
            .format((i+1), int(np.log10(nrange)+1), nrange,
                    test_block[ii,0], test_block[ii,1]))

    test_block_i = test_block[ii]
    test_outp_i = test_outp[ii]
    test_ske_i = test_ske[ii]
    test_FGPA_i = test_FGPA[ii]
    test_DM_i = DM_general[0, test_block_i[0], test_block_i[1], :].numpy()

    stat_i = test_accuracy(test_block_i, test_outp_i, test_ske_i, test_FGPA_i,
                      test_DM_i, F_mean, v_end, folder_outp, bins, plot=True)
    accuracy_i, rela_err_i = stat_i[[4,5]]
    accuracy.update(accuracy_i, 1)
    rela_err.update(rela_err_i, 1)
    accu_arr[ii] = accuracy_i
    erro_arr[ii] = rela_err_i
    oneDPS[:,ii] = stat_i[0], stat_i[1], stat_i[2], stat_i[3]

print('Measuring accuracy of the rest skewers...')
for i, ii in enumerate(test_sp2):

    test_block_i = test_block[ii]
    test_outp_i = test_outp[ii]
    test_ske_i = test_ske[ii]
    test_FGPA_i = test_FGPA[ii]
    test_DM_i = DM_general[0, test_block_i[0], test_block_i[1], :].numpy()

    stat_i = test_accuracy(test_block_i, test_outp_i, test_ske_i, test_FGPA_i,
                      test_DM_i, F_mean, v_end, folder_outp, bins, plot=False)
    accuracy_i, rela_err_i = stat_i[[4,5]]
    accuracy.update(accuracy_i, 1)
    rela_err.update(rela_err_i, 1)
    accu_arr[ii] = accuracy_i
    erro_arr[ii] = rela_err_i
    oneDPS[:,ii] = stat_i[0], stat_i[1], stat_i[2], stat_i[3]


print('Plotting average 1DPS and PDF...')
oneDPS_backup = oneDPS
oneDPS = oneDPS.mean(axis=1)
oneDPS = oneDPS[~np.isnan(oneDPS)].reshape(4,-1)
bln = oneDPS[0]<0.1
accuracy_gen = np.abs((oneDPS[1]-oneDPS[2])/oneDPS[2])[bln].mean()
rela_err_gen = np.abs((oneDPS[1]-oneDPS[2])/oneDPS[2])[bln].std()
accuracy_FGPA = np.abs((oneDPS[3]-oneDPS[2])/oneDPS[2])[bln].mean()
rela_err_FGPA = np.abs((oneDPS[3]-oneDPS[2])/oneDPS[2])[bln].std()

outp_hist, F_hist = np.histogram(test_outp, bins=np.arange(0,1.05,0.05))
test_hist, F_hist = np.histogram(test_ske,  bins=np.arange(0,1.05,0.05))
FGPA_hist, F_hist = np.histogram(test_FGPA, bins=np.arange(0,1.05,0.05))
outp_hist = np.append(outp_hist, outp_hist[-1]) / len(test_ske)
test_hist = np.append(test_hist, test_hist[-1]) / len(test_ske)
FGPA_hist = np.append(FGPA_hist, FGPA_hist[-1]) / len(test_ske)
bln = (F_hist>=0.1) & (F_hist<0.9)
accuracy_hist = np.abs((outp_hist-test_hist)/test_hist)[bln].mean()
rela_err_hist = np.abs((outp_hist-test_hist)/test_hist)[bln].std()
m_hist_FGPA = np.abs((FGPA_hist-test_hist)/test_hist)[bln].mean()
s_hist_FGPA = np.abs((FGPA_hist-test_hist)/test_hist)[bln].std()

In [ ]:

fig, axes = plt.subplots(2,2,figsize=(15,11))

p0 = axes[0,0].hist(accu_arr[:,0], label='ML', bins=np.arange(0, 1.7, 0.1))
axes[0,0].set_xlim(-0.1, 1.8)
axes[0,0].set_ylim(axes[0,0].get_ylim())
p1 = axes[0,0].vlines(x=np.median(accu_arr[:,0]), ymin=0, ymax=9999, linestyle='--')
p9 = axes[0,0].hist(accu_arr[:,1], label='FGPA', color='tab:green', bins=np.arange(0, 1.7, 0.1), alpha=0.5)
axes[0,0].set_xlabel('accuracy $m$', fontsize=14)
axes[0,0].set_title('pdf of $m$', fontsize=14)
axes[0,0].tick_params(labelsize=12, direction='in')
customs = [p1, 
          Line2D([0], [0], marker='o', color='w',
                  markerfacecolor='k', markersize=5),
          p9]
axes[0,0].legend(customs, ['median $m=%.4f$'%np.median(accu_arr[:,0]),
                        '$N=%d$'%len(accu_arr)], fontsize=12, loc=1)

axes[0,1].hist(erro_arr[:,0], color='grey', bins=np.arange(0, 1.7, 0.1))
axes[0,1].set_xlim(-0.1, 1.8)
axes[0,1].set_ylim(axes[0,1].get_ylim())
p2 = axes[0,1].vlines(x=np.median(erro_arr[:,0]), ymin=0, ymax=9999, linestyle='--')
axes[0,1].set_xlabel('error $s$', fontsize=14)
axes[0,1].set_title('pdf of $s$', fontsize=14)
axes[0,1].tick_params(labelsize=12, direction='in')
customs = [p2, 
          Line2D([0], [0], marker='o', color='w',
                  markerfacecolor='k', markersize=5)]
axes[0,1].legend(customs, ['median $s=%.4f$'%np.median(erro_arr[:,0]),
                        '$N=%d$'%len(erro_arr)], fontsize=12, loc=1)

p3, = axes[1,0].plot(oneDPS[0], oneDPS[1], label='ML')
p4, = axes[1,0].plot(oneDPS[0], oneDPS[2], label='Hydro', alpha=0.5)
p7, = axes[1,0].plot(oneDPS[0], oneDPS[3], label='FGPA', alpha=0.5)
axes[1,0].set_xlabel(r'$k\ (\mathrm{s/km})$', fontsize=14)
axes[1,0].set_ylabel(r'$kP_\mathrm{1D}/\pi$', fontsize=14)
axes[1,0].set_xscale('log')
axes[1,0].set_yscale('log')
axes[1,0].set_ylim(axes[1,0].get_ylim())
axes[1,0].vlines(x=0.1, ymin=1e-8, ymax=1e8)
axes[1,0].set_title('Average 1DPS', fontsize=14)
axes[1,0].tick_params(labelsize=12, direction='in', which='both')
customs = [p3,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p7,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p4]
axes[1,0].legend(customs, [p3.get_label(), '$m=%.3f$'%accuracy_gen, '$s=%.3f$'%rela_err_gen, 
                    p7.get_label(),'$m=%.3f$'%accuracy_FGPA, '$s=%.3f$'%rela_err_FGPA,
                    p4.get_label()], fontsize=12)

p5, = axes[1,1].step(F_hist, outp_hist, where='post', label='ML')
p6, = axes[1,1].step(F_hist, test_hist, where='post', label='Hydro', alpha=0.5)
p8, = axes[1,1].step(F_hist, FGPA_hist, where='post', label='FGPA', alpha=0.5)
axes[1,1].set_xlabel(r'$F$', fontsize=18)
axes[1,1].set_ylabel(r'Counts', fontsize=18)
axes[1,1].set_xlim([0, 1])
axes[1,1].set_ylim(bottom=0)
axes[1,1].vlines(x=0.1, ymin=0, ymax=6000)
axes[1,1].vlines(x=0.9, ymin=0, ymax=6000)
axes[1,1].set_title('Average PDF of $F$', fontsize=14)
axes[1,1].tick_params(labelsize=12, direction='in')
customs = [p5,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p8,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p6]
axes[1,1].legend(customs, [p5.get_label(), '$m=%.3f$'%accuracy_hist, '$s=%.3f$'%rela_err_hist,
                    p8.get_label(),'$m=%.3f$'%m_hist_FGPA, '$s=%.3f$'%s_hist_FGPA,
                    p6.get_label()], fontsize=12)

#plt.savefig(folder_outp / ('average.png'), dpi=300, bbox_inches='tight') 

In [ ]:
from scipy import optimize
def norm_specs(skewers,redshift):
    tauevo = 0.001845*(1+redshift)**3.924 #optical depth corresponding to the mean-flux of Faucher-Giguere et al. 2008 
    fobs = np.exp(-tauevo)
    def fun(a): return np.mean(np.exp(-a*skewers)-fobs)
    a = optimize.newton(fun, 1)    
    return a
rs = 2.0020281392528516 #redshift of the snapshot


# load model
print('Loading model...')
model = get_residual_network().float().to(device)
model.load_state_dict(torch.load('params/params_%s.pkl'\
        %time.strftime("%Y-%m-%d_%H:%M:%S", localtime)))
'''
model.load_state_dict(torch.load('params/HyPhy_%s'\
        %time.strftime("%Y-%m-%d_%H:%M:%S", localtime)))
'''


# loss
criterion = nn.L1Loss()


# record starr time
start_time = time.time()


# start test
print('Begin testing...')
test_outp, test_losses = test(test_ske, test_block, DM_general, DM_param,
                        test_batch, train_insize, model, criterion, device, start_time)

print("Test Summary: ")
print("\tTest loss: {}".format(test_losses))

# restore test skewers
print('Restoring test skewers...')
nz = (ske_len/train_ousize[2]).astype('int')
test_outp = test_outp.reshape(-1, nz, train_ousize[0],
                            train_ousize[1], train_ousize[2])\
                            .transpose(0, 2, 3, 1, 4).reshape(-1, ske_len)
test_ske = test_ske.numpy().reshape(-1, nz, train_ousize[0],
                            train_ousize[1], train_ousize[2])\
                            .transpose(0, 2, 3, 1, 4).reshape(-1, ske_len)
test_FGPA = test_FGPA.numpy().reshape(-1, nz, train_ousize[0],
                            train_ousize[1], train_ousize[2])\
                            .transpose(0, 2, 3, 1, 4).reshape(-1, ske_len)

test_outp = toF_proc(test_outp)
test_ske  = toF_proc(test_ske)
test_FGPA = toF_proc(test_FGPA)

test_coord = test_block.reshape(-1, nz, 3)[:, 0, 0:2].T.reshape(2, 1, 1, -1)
xcoor = np.arange(train_ousize[0]) - np.arange(train_ousize[0]).mean()
ycoor = np.arange(train_ousize[1]) - np.arange(train_ousize[1]).mean()
mesh  = np.expand_dims(np.array(np.meshgrid(xcoor, ycoor)), -1)
test_block = (test_coord + mesh).transpose(3,2,1,0).reshape(-1, 2).astype(int)
del test_coord, xcoor, ycoor, mesh


print('Plotting example skewers...')
# generate comparison images
folder_outp = Path.cwd()/'test_figs'/('%s_x'\
        %time.strftime("%Y-%m-%d_%H:%M:%S_FGPA", localtime))
if not os.path.exists(folder_outp):
    os.makedirs(folder_outp)


from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
z = 2.0
h= 0.704
cosmo = FlatLambdaCDM(H0=100.0*h, Om0=0.2726, Ob0=0.0456)
Hz = cosmo.H(z)
a = 1.0 / (1.0 + z)
v_end  = (a * Hz * 75/h*u.Mpc).to(u.km / u.s).value
F_mean = np.array([test_ske.mean(), test_outp.mean(), test_FGPA.mean()])
print(F_mean)

nrange = min(len(test_ske), 50)
test_sp = np.arange(len(test_ske))
np.random.seed(99)
np.random.shuffle(test_sp)
test_sp1 = test_sp[:int(nrange)].astype('int')
test_sp2 = test_sp[int(nrange):].astype('int')

bins = int(15)
accuracy = AverageMeter()
rela_err = AverageMeter()
accu_arr = np.zeros(len(test_ske))
erro_arr = np.zeros(len(test_ske))
oneDPS   = np.zeros(shape=(4, len(test_ske), bins))


# loop
for i, ii in enumerate(test_sp1):
    print('Plotting {:{}d}/{}, y{:03d}z{:03d}.png...'\
            .format((i+1), int(np.log10(nrange)+1), nrange,
                    test_block[ii,0], test_block[ii,1]))

    test_block_i = test_block[ii]
    test_outp_i = test_outp[ii]
    test_ske_i = test_ske[ii]
    test_FGPA_i = test_FGPA[ii]
    test_DM_i = DM_general[0, test_block_i[0], test_block_i[1], :].numpy()

    stat_i = test_plot(test_block_i, test_outp_i, test_ske_i, test_FGPA_i,
                      test_DM_i, F_mean, v_end, folder_outp, bins)
    accuracy_i, rela_err_i = stat_i[[4,5]]
    accuracy.update(accuracy_i, 1)
    rela_err.update(rela_err_i, 1)
    accu_arr[ii] = accuracy_i
    erro_arr[ii] = rela_err_i
    oneDPS[:,ii] = stat_i[0], stat_i[1], stat_i[2], stat_i[3]

print('Measuring accuracy of the left skewers...')
for i, ii in enumerate(test_sp2):

    test_block_i = test_block[ii]
    test_outp_i = test_outp[ii]
    test_ske_i = test_ske[ii]
    test_FGPA_i = test_FGPA[ii]
    test_DM_i = DM_general[0, test_block_i[0], test_block_i[1], :].numpy()

    stat_i = test_accuracy(test_block_i, test_outp_i, test_ske_i, test_FGPA_i,
                          F_mean, v_end, folder_outp, bins)
    accuracy_i, rela_err_i = stat_i[[4,5]]
    accuracy.update(accuracy_i, 1)
    rela_err.update(rela_err_i, 1)
    accu_arr[ii] = accuracy_i
    erro_arr[ii] = rela_err_i
    oneDPS[:,ii] = stat_i[0], stat_i[1], stat_i[2], stat_i[3]


print('Plotting average 1DPS and PDF...')
#oneDPS_backup = oneDPS
oneDPS = oneDPS.mean(axis=1)
oneDPS = oneDPS[~np.isnan(oneDPS)].reshape(4,-1)
bln = oneDPS[0]<0.1
accuracy_gen = np.abs((oneDPS[1]-oneDPS[2])/oneDPS[2])[bln].mean()
rela_err_gen = np.abs((oneDPS[1]-oneDPS[2])/oneDPS[2])[bln].std()
accuracy_FGPA = np.abs((oneDPS[3]-oneDPS[2])/oneDPS[2])[bln].mean()
rela_err_FGPA = np.abs((oneDPS[3]-oneDPS[2])/oneDPS[2])[bln].std()

outp_hist, F_hist = np.histogram(test_outp, bins=np.arange(0,1.05,0.05))
test_hist, F_hist = np.histogram(test_ske,  bins=np.arange(0,1.05,0.05))
FGPA_hist, F_hist = np.histogram(test_FGPA, bins=np.arange(0,1.05,0.05))
outp_hist = np.append(outp_hist, outp_hist[-1]) / len(test_ske)
test_hist = np.append(test_hist, test_hist[-1]) / len(test_ske)
FGPA_hist = np.append(FGPA_hist, FGPA_hist[-1]) / len(test_ske)
bln = (F_hist>=0.1) & (F_hist<0.9)
accuracy_hist = np.abs((outp_hist-test_hist)/test_hist)[bln].mean()
rela_err_hist = np.abs((outp_hist-test_hist)/test_hist)[bln].std()
m_hist_FGPA = np.abs((FGPA_hist-test_hist)/test_hist)[bln].mean()
s_hist_FGPA = np.abs((FGPA_hist-test_hist)/test_hist)[bln].std()


fig, axes = plt.subplots(2,2,figsize=(15,11))

p0=axes[0,0].hist(accu_arr, color='grey', bins=np.arange(0, 1.7, 0.1))
axes[0,0].set_xlim(-0.1, 1.8)
axes[0,0].set_ylim(axes[0,0].get_ylim())
p1 = axes[0,0].vlines(x=accu_arr.mean(), ymin=0, ymax=9999, linestyle='--')
axes[0,0].set_xlabel('accuracy $m$', fontsize=14)
axes[0,0].set_title('pdf of $m$', fontsize=14)
axes[0,0].tick_params(labelsize=12, direction='in')
customs = [p1, 
          Line2D([0], [0], marker='o', color='w',
                  markerfacecolor='k', markersize=5)]
axes[0,0].legend(customs, ['average $m=%.4f$'%accu_arr.mean(),
                        '$N=%d$'%len(accu_arr)], fontsize=12, loc=1)

axes[0,1].hist(erro_arr, color='grey', bins=np.arange(0, 1.7, 0.1))
axes[0,1].set_xlim(-0.1, 1.8)
axes[0,1].set_ylim(axes[0,1].get_ylim())
p2 = axes[0,1].vlines(x=erro_arr.mean(), ymin=0, ymax=9999, linestyle='--')
axes[0,1].set_xlabel('error $s$', fontsize=14)
axes[0,1].set_title('pdf of $s$', fontsize=14)
axes[0,1].tick_params(labelsize=12, direction='in')
customs = [p2, 
          Line2D([0], [0], marker='o', color='w',
                  markerfacecolor='k', markersize=5)]
axes[0,1].legend(customs, ['average $s=%.4f$'%erro_arr.mean(),
                        '$N=%d$'%len(erro_arr)], fontsize=12, loc=1)

p3, = axes[1,0].plot(oneDPS[0], oneDPS[1], label='Predicted')
p4, = axes[1,0].plot(oneDPS[0], oneDPS[2], label='Real', alpha=0.5)
p7, = axes[1,0].plot(oneDPS[0], oneDPS[3], label='FGPA', alpha=0.5)
axes[1,0].set_xlabel(r'$k\ (\mathrm{s/km})$', fontsize=14)
axes[1,0].set_ylabel(r'$kP_\mathrm{1D}/\pi$', fontsize=14)
axes[1,0].set_xscale('log')
axes[1,0].set_yscale('log')
axes[1,0].set_ylim(axes[1,0].get_ylim())
axes[1,0].vlines(x=0.1, ymin=1e-8, ymax=1e8)
axes[1,0].set_title('Average 1DPS', fontsize=14)
axes[1,0].tick_params(labelsize=12, direction='in', which='both')
customs = [p3,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p7,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p4]
axes[1,0].legend(customs, [p3.get_label(), '$m=%.3f$'%accuracy_gen, '$s=%.3f$'%rela_err_gen, 
                    p7.get_label(),'$m=%.3f$'%accuracy_FGPA, '$s=%.3f$'%rela_err_FGPA,
                    p4.get_label()], fontsize=12)

p5, = axes[1,1].step(F_hist, outp_hist, where='post', label='Predicted')
p6, = axes[1,1].step(F_hist, test_hist, where='post', label='Real', alpha=0.5)
p8, = axes[1,1].step(F_hist, FGPA_hist, where='post', label='FGPA', alpha=0.5)
axes[1,1].set_xlabel(r'$F$', fontsize=18)
axes[1,1].set_ylabel(r'Counts', fontsize=18)
axes[1,1].set_xlim([0, 1])
axes[1,1].set_ylim(bottom=0)
axes[1,1].vlines(x=0.1, ymin=0, ymax=6000)
axes[1,1].vlines(x=0.9, ymin=0, ymax=6000)
axes[1,1].set_title('Average PDF of $F$', fontsize=14)
axes[1,1].tick_params(labelsize=12, direction='in')
customs = [p5,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p8,
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          Line2D([0], [0], marker='o', color='w',
                      markerfacecolor='k', markersize=5),
          p6]
axes[1,1].legend(customs, [p5.get_label(), '$m=%.3f$'%accuracy_hist, '$s=%.3f$'%rela_err_hist,
                    p8.get_label(),'$m=%.3f$'%m_hist_FGPA, '$s=%.3f$'%s_hist_FGPA,
                    p6.get_label()], fontsize=12)

#plt.savefig(folder_outp / ('average.png'), dpi=300, bbox_inches='tight') 

In [ ]:
# this is unnormalized